In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import math
# import scipy.sparse as sps

In [ ]:
######################################################################################
#################### Verouderde code #################################################
######################################################################################

print('Deze code is verouderd! Gebruik het dus niet.')

path='/usr/people/out/Documents/220kV_Noord'
kabels=pd.read_csv(path+'/spreadsheetcables.csv', delimiter = ';')
trafo=pd.read_csv(path+'/spreadsheettrafo.csv', delimiter = ';')

###### make some matrices #######
trafo_connect=np.zeros((len(trafo),len(trafo))) #connectivity trafo
cables_connect=np.zeros((len(kabels),len(kabels))) #connectivity cables
cable_to_trafo=np.zeros((len(trafo),len(kabels))) #direct connection trafo cable
trafo_all_connections=np.zeros((len(trafo),len(kabels))) #connections possible between trafo and every cable
trafo_cond=np.zeros((len(trafo),len(trafo))) # The conductivity matrix

kabel_resist=np.zeros(len(kabels))
kabel_cond=np.zeros(len(kabels)) #gives total conductivity per connected cable segment per cable
total_kabel_cond=np.zeros(len(kabels)) #gives total conductivity per segment for all cables
list_of_resist=np.zeros(len(kabels)) #resistance per cabletraject
aantal_kabels=np.zeros(len(kabels)) #aantal kabels (per fase) per traject


##### make trafo-trafo connectivity matrix ###### CORRECT
for line in range(len(trafo)): 
    temp=trafo.at[line,'verbonden trafo'] #get right column
    temp=temp.split(",") #split values
#     trafo_connect[line,line]=True #trafo itself is ofcourse connected
    for item in temp:
        temp2=int(item)
        trafo_connect[line,temp2-1]=True #check for connection other trafo
        del temp2
    del temp
    
    
###### make trafo-cable connectivity matrix ###### CORRECT
#direct connection
for line in range(len(trafo)):
    temp=trafo.at[line,'aansluitingen']
    temp=temp.split(",")
    for item in temp:
        temp2=int(item)
        cable_to_trafo[line,temp2-1]=True
        del temp2
    del temp  

#all connection
for line in range(len(trafo)):
    temp=trafo.at[line,'alle aansluitingen']
    temp=temp.split(",")
    for item in temp:
        temp2=int(item)
        trafo_all_connections[line,temp2-1]=True
        del temp2
    del temp

    
###### make cable-cable connectivity matrix ###### CORRECT
kabels=kabels.fillna(0) #make cables matrix and fill NaN with 0
for line in range(len(kabels)):
    temp=kabels.at[line,'verbonden']
    cables_connect[line,line]=True #a cable is connected to itself:)
    
    if temp!=0:
        temp=temp.split(",")

        for item in temp: #find other connections
            temp2=int(item)
            cables_connect[line,temp2-1]=True
            del temp2
    del temp 
    
    
###### make total conductivity vector for cables####### CORRECT
for line in range(len(kabels)):
    list_of_resist[line]=float(1./kabels.at[line,'conductivity'])

# now every cable has the conductivity of everything that is connected to it
kabel_resist=np.matmul(cables_connect,list_of_resist)
kabel_cond=1./kabel_resist

for line in range(len(kabels)):
    aantal_kabels[line]=kabels.at[line,'kab/3'] #get amount of cables per traject
    total_kabel_cond[line]=aantal_kabels[line]*kabel_cond[line]
print(trafo_connect)


##### now conductivity matrix #####
# first: trace
for line in range(len(trafo)):
    temp=np.matmul(cable_to_trafo[line,:],total_kabel_cond)
    trafo_cond[line,line]=temp+trafo.at[line,'conductivity total']
    del temp
    
# then the off-diagonal components 
for row,line in enumerate(trafo_connect):
    for column,item in enumerate(line):
        if item:
            temp=trafo_all_connections[row,:]+trafo_all_connections[column,:]
            for counter,value in enumerate(temp):
                if value == 2:
                    break
            trafo_cond[row,column]=-total_kabel_cond[counter]
            del temp
print(trafo_cond)


#       trafo_cond[row,column]=-cable_to_trafo[column]

# print(trafo_cond)
# trafo_cond=np.zeros((len(trafo),len(trafo))) # determine conductivity between trafo
# for line in range(len(trafo))